In [ ]:
# Quick implementation of a punch and fill algorithm to remove bragg peaks

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, autojit, prange
from scipy.ndimage.filters import convolve
# Disable inline plots
#%matplotlib qt

In [17]:
# Pad dataset before filter based on window provided
# Give data set (d_set), and window width (x)
# Needs numpy as np
@jit
def winpad(d_set, x):
    if (x % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    rad = np.int(np.floor(x/2)) # Calculate radius of window given
    pad = np.pad(d_set, rad, 'edge')
    return pad

# Does a 1D ft, doesn't need to be rolled over
@jit
def ft(signal):
    return np.fft.fftshift(np.fft.fftn(np.fft.ifftshift(signal)))
# Fourier transform 2D slices of an image
@jit
def ft_slices(data):
    shape = np.shape(data) # Get shape of input data
    fslices = np.zeros(shape, dtype = np.complex_) # Array of complex zeros with same shape as input data
    l1 = shape[0] # Number of 2D slices through 3D volume
    for i in range (0, l1):
        fslices[i] = ft(data[i]) # Assign FT of individual slices to corresponding slice in zero array
        #print(i, end = ' ') # Testing counter printout
    return fslices
# Take FT of columns through 2D slices
@jit
def ft_columns(data):
    shape = np.shape(data) # Get shape of input data
    out = np.zeros(shape, dtype = np.complex_) # Array of complex zeros with same shape as input data
    l1 = shape[0] # Total number of loops for y dim
    l2 = shape[1] # Total number of loops for z dim
    for i in range (0, l1): # Loop through y
        for j in range (0, l2): # Loop through z
            out[:, i, j] = ft(data[:, i, j]) # Assign FT through x axis to corresponding region of zero array
        #print (i, end = ' ') # Testing counter printout
    return out
# Take 3D FT by taking FT of 2D slices through an image and then 1D columns perpendicular to slices
@jit
def ft_3d(data):
    slices = ft_slices(data)
    cols = ft_columns(slices)
    return cols
# Generates a gaussian window centered in the image with given input sigma
# Pixels in image are mapped from -2 to 2
# Does not drop off to 0 at edges, maybe fix, probably doesn't actually matter
def gauss_win_3d(width, sigma):
    rng = 2
    x, y, z = np.meshgrid(np.linspace(-rng,rng,width), np.linspace(-rng,rng,width), np.linspace(-rng,rng,width))
    d = np.sqrt(x*x+y*y+z*z) # generate matrix of distances
    mu = 0 # Make gaussian centered in image
    g = np.exp(-( (d-mu)**2 / ( 2.0 * sigma**2 ) ) ) # Map gaussian over distance matrix
    #cent = np.int(np.ceil(width/2)) # Get center row 
    #minval = np.min(g[cent]) # Get minimum value from center row of image
    #g = np.subtract(g, minval) # Subtract minimum value from gaussian image to shift ends to zero
    #g[g < 0] = 0 # Set any negative values in gaussian mask to 0
    
    return g
# Punch out bragg peaks by setting value to 0
# Bragg peaks are located as outliers in some moving window
from numba import autojit, prange
@jit(parallel=True)
def punch(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    winrad = np.floor(np.divide(width, 2)) # Window radius
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mean = np.mean(np.concatenate((dat[0:winrad], dat[winrad+1:]))) # Get mean of input data set without center value
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
        else:
            out[x, y, z] = 0 # Set value equal to zero if outlier detected
            continue
    
    return out

In [14]:
# Create gaussian window
# g = gauss_win_3d(501, 0.85)
g = np.load('gaussian window function.npy')
# Load diffraction data
pmn = np.load('PMN_normdata_48sym_300K.npy')
# Multiply by window function
pmn = np.multiply(pmn, g) 
del g

In [15]:
# Punch out bragg peaks
pdat = punch(pmn, 5)

In [21]:
# Gaussian convolution kernel
gk = gauss_win_3d(10, 1)
# Convolve data set with removed bragg peaks with gaussian
cdat = convolve(pdat, gk)

In [39]:
# Show slice of 3D volume
plt.imshow(cdat[250])

In [40]:
# Plot line through removed bragg peaks in slice of 3d volume
plt.plot(cdat[250][270])

In [27]:
# 3D fourier transform of data 
ft1 = ft_3d(cdat)

In [28]:
# Generate mask to remove center bright point in data set 
mask = np.ones((501,501, 501))
mask[248:253, 248:253, 248:253] = 0
# Multiply data set by mask 
ft1 = np.multiply(ft1, mask)

In [42]:
# Show slice through 3D PDF
plt.imshow(np.real(ft1)[250], cmap = 'coolwarm')
plt.title('punch and fill')
plt.colorbar()